# SpaceRocks

### Vectorized coordinate transformation and ephemeris calculation with robust unit handling.

To install, simply `pip install spacerocks`

In [1]:
from spacerocks import SpaceRock, Propagate
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

You can also pass in just a single object or an array of objects. I'll try BP519. I specified an observatory code, so a topocentric correction will be applied to the Earth's position.

In [2]:
df = pd.read_json('/Users/kjnapier/Desktop/mpcorb_extended.json')
df = df[df.Orbit_type == 'Jupiter Trojan']
df = df[df.Num_opps > 4]

In [3]:
rocks = SpaceRock(a=df.a.values, 
                  e=df.e.values, 
                  inc=df.i.values, 
                  arg=df.Peri.values, 
                  node=df.Node.values, 
                  t_peri=df.Tp.values, 
                  obsdate=df.Epoch.values,
                  H=df.H.values, 
                  name=df.Principal_desig.values.astype(str),
                  input_coordinates='keplerian',
                  input_frame='heliocentric',
                  input_angles='degrees',
                  obscode=None,
                  calc_equa=True)

In [4]:
futobs = Propagate(rocks, obsdates=np.array([2459003.5, 2459004.5]), model=5, gr=False, add_pluto=False)

In [5]:
futobs.ra

<Angle [0.3999566 , 3.66422869, 0.2200105 , ..., 4.47263397, 5.95724854,
        0.47034673] rad>

In [4]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [5]:
@interact
def sample_interactive_plot(max_inc=(0, 90, 1)):
    p = rocks[rocks.inc.deg < max_inc]
    return p.plot_radec()

interactive(children=(IntSlider(value=45, description='max_inc', max=90), Output()), _dom_classes=('widget-int…